In [6]:
import pandas as pd
import numpy as np
import pickle


In [2]:
# Definimos path
pathdata = '/home/daniu/Documentos/fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/Users/danielarisaro/Documents/Fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/'

df_people_pool = pd.read_csv(pathdata + 'data_output/Base_pool_individuos_solo_con_replicas_actuales.csv', low_memory=False, index_col=0)


In [10]:
with open(pathdata + 'modulos/diccionarios/' + 'diccionario_provincia_region.pickle', 'rb') as file:
    map_provincia_region = pickle.load(file)

with open(pathdata + 'modulos/diccionarios/' + 'diccionario_provincia.pickle', 'rb') as file:
    dict_cod_provincia = pickle.load(file)

In [33]:
del df_ocupados

In [39]:
df_temp = df_people_pool.query('CH06 >= 14')

df_ocupados = df_temp[(df_temp['P21_new']!=-9) & (df_temp['ESTADO']==1) & \
                      (df_temp['CAT_OCUP'].isin([2,3]))][['PROVINCIA', 'PONDIIO_new', 'P21_new', 'CH04']]
df_ocupados = df_ocupados[df_ocupados['P21_new']>0]
df_ocupados['Multiplication'] = df_ocupados['P21_new'] * df_ocupados['PONDIIO_new']

df_ocupados['PROVINCIA'] = df_ocupados['PROVINCIA'].map(dict_cod_provincia)
df_ocupados['PROVINCIA'] = df_ocupados['PROVINCIA'].replace('CABA', 'Ciudad Autónoma de Buenos Aires')
df_ocupados['PROVINCIA'] = df_ocupados['PROVINCIA'].replace('TdF', 'Tierra del Fuego, Antártida e Islas del Atlántico Sur')

df_ocupados['REGION'] = df_ocupados['PROVINCIA'].map(map_provincia_region)

numerador = df_ocupados.groupby(['CH04', 'REGION'])['Multiplication'].sum().unstack(level=0)
denominador = df_ocupados.groupby(['CH04', 'REGION'])['PONDIIO_new'].sum().unstack(level=0)

tasa = numerador.div(denominador, fill_value=np.nan)    
tasa = tasa.rename(columns= {1:'Varon', 2:'Mujer'})

ratio = (tasa['Mujer'] / tasa['Varon']).to_frame()
ratio.rename(columns={0: 'Ingreso laboral promedio'}, inplace=True)

In [42]:
ratio.to_clipboard

,Ingreso laboral promedio
REGION,
Cuyo,0.741476
Noreste,0.828135
Noroeste,0.766768
Pampeana,0.745097
Patagonia,0.748317


In [14]:
df_ocupados['PROVINCIA']

220                     CABA
222                     CABA
400                     CABA
402                     CABA
404                     CABA
                ...         
79296    Santiago del Estero
79297    Santiago del Estero
79298    Santiago del Estero
79300               San Juan
79301               San Juan
Name: PROVINCIA, Length: 44202, dtype: object

In [ ]:



    size = df_estado[(df_estado[p21]>0) & (df_estado[p21]!=np.nan)].groupby(['CH04', var]).size().unstack(level=0)
    size = size.rename(columns= {1:'N_v', 2:'N_m'})

    n_pob = df_ocupados.groupby(['CH04', var])[pondera].sum().unstack(level=0)
    n_pob = n_pob.rename(columns= {1:'N_pob_v', 2:'N_pob_m'})

    size[['N_pob_v', 'N_pob_m']] = n_pob[['N_pob_v', 'N_pob_m']]
    size['N_pob_tot'] = n_pob['N_pob_v'] + n_pob['N_pob_m']    

    df_ocupados['Deviation'] = df_ocupados[p21] - df_ocupados.groupby(['CH04', var])[p21].transform('mean')
    df_ocupados['Weighted_Deviation'] = df_ocupados[pondera] * df_ocupados['Deviation']**2
    weighted_variance = df_ocupados.groupby(['CH04', var])['Weighted_Deviation'].sum() / df_ocupados.groupby(['CH04', var])[pondera].sum()

    weighted_std = np.sqrt(weighted_variance.astype('float')).unstack(level=0)
    weighted_std = weighted_std.rename(columns= {1:'Varon', 2:'Mujer'})

    standard_error = np.sqrt((weighted_std['Mujer']**2 / size['N_m']) + (weighted_std['Varon']**2 / size['N_v']))

    degrees_of_freedom = size['N_m'] + size['N_v'] - 2
    t = stats.t.ppf((1 + confidence_level) / 2, df=degrees_of_freedom)
    margin_of_error = t * standard_error
    
    ratio = (tasa['Mujer'] / tasa['Varon']).to_frame()
    ratio.rename(columns={0: 'Ingreso laboral promedio'}, inplace=True)

    lower_bound = tasa['Mujer'] / tasa['Varon'] - margin_of_error
    upper_bound = tasa['Mujer'] / tasa['Varon'] + margin_of_error
    relative_standard_error = margin_of_error / (tasa['Mujer'] / tasa['Varon'])

    error = pd.concat([lower_bound, upper_bound, margin_of_error, relative_standard_error], axis=1)
    error.columns = ['LI', 'LS', 'ME', 'ER']
    error[['LI', 'LS', 'ME', 'ER']] = pd.NaT

    return tasa, size, ratio*100, error